# Setup - General

In [ ]:
# Set up environment and define shape of data
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

pd.set_option('display.max_colwidth', None)

# List of columns to analyze
# Note that pandas will make any duplicate column names unique by appending a sequence number
# to subsequent occurences of them when importing said data
# (e.g. HMS Rank, HMS Rank.1, HMS Rank.2, etc.)
SCHEMA = {
    '1. Sex assigned at birth': 'string',
    '2. Gender Identity': 'string',
    'Black or African American': 'string',
    'Hispanic or Latino': 'string',
    'American Indian or Alaskan Native': 'string',
    'Native Hawaiian and other Pacific Islander': 'string',
    'Cambodian or Laotian': 'string',
    'None': 'string',
    'Prefer not to answer': 'string',
    '4. Identification as ': 'string',
    '5. Requirement for workplace accommodations: (Reasonable accommodations in the ' + \
    'workplace support people with disability and/or chronic health conditions in ' + \
    'performing their jobs.) If you would like more information about obtaining ' + \
    'accommodations, contact MGH OHS.': 'string',
    '1. Are you considering or engaging in the process of retirement? ': 'string',
    '2. Are you interested in learning more about HMS promotions and discussing advancement ' + \
    'on the HMS ladder? ': 'string',
    'MD': 'string',
    'MBChB': 'string',
    'DO': 'string',
    'PhD': 'string',
    'DPhil': 'string',
    'Sci D': 'string',
    'RDN': 'string',
    'MS': 'string',
    'MSN': 'string',
    'MPH': 'string',
    'MPA': 'string',
    'MBA': 'string',
    'Other': 'string',
    'i. HMS Rank': 'string',
    'ii. Date (yyyy)': 'string',
    'i. HMS Rank.1': 'string',
    'ii. Date (yyyy).1': 'string',
    'i. HMS Rank.2': 'string',
    'ii. Date (yyyy).2': 'string',
    'i. HMS Rank.3': 'string',
    'ii. Date (yyyy).3': 'string',
    'i. HMS Rank.4': 'string',
    'ii. Date (yyyy).4': 'string',
    'i. Job Title': 'string',
    'ii. Date (yyyy).5': 'string',
    'i. Job Title.1': 'string',
    'ii. Date (yyyy).6': 'string',
    'i. Job Title.2': 'string',
    'ii. Date (yyyy).7': 'string',
    'i. Job Title.3': 'string',
    'ii. Date (yyyy).8': 'string',
    'i. Job Title.4': 'string',
    'ii. Date (yyyy).9': 'string',
}

# Column label capitalization is inconsistent between data sets
# Convert all names to lowercase
SCHEMA = {key.lower(): val for key, val in SCHEMA.items()}

# Store a list of column labels for easier reference
COLUMNS = list(SCHEMA.keys())

# Create scaffolding for DataFrame that will house concatenated datasets
df = pd.DataFrame(columns=COLUMNS).astype(SCHEMA)

# Helper for percentage formula
# Default denominator is the total number of faculty invited to complete the survey
INVITED = 1095
def calcPercent(numerator, denominator=INVITED):
    return round(numerator / denominator * 100, 2)

# Bar graph bar colors
BAR_COLORS = ['red','blue','gray', 'pink', 'purple', 'yellow', 'green', 'orange']

# Create and open a file for dumping data
DUMP = pd.ExcelWriter('DataDump.xlsx')

In [ ]:
# Load all datasets into memory

from pathlib import Path

for filepath in Path("./data").iterdir():
    if filepath.is_file():
        print(f"Importing {filepath}")
        rawDf = pd.read_excel(
            filepath,
            header=0,
            engine="openpyxl"
        )
        # Convert raw column names to lowercase
        rawDf.rename(columns=str.lower, inplace=True)
        # Select only the COLUMNS we are interested in
        rawDf = rawDf.filter(items=COLUMNS)
        # Clear out blank rows
        trimmedDf = rawDf.dropna(how='all')
        print(
            f"{rawDf.shape[0] - trimmedDf.shape[0]} blank rows dropped.  " + \
            f"Remaining rows: {trimmedDf.shape[0]}.\n"
        )
        # Append rows to main DataFrame
        initial = df.shape[0]
        df = pd.concat([df, trimmedDf], ignore_index=True)
        del rawDf, trimmedDf, initial

del Path, filepath

<hr style="border:3px solid gray">

# Individual datapoint analysis

# Dataset Size

In [ ]:
print(
    f"{df.shape[0]} faculty provided at least some demographic or rank data for a " + \
    f"{calcPercent(df.shape[0])}% overall response rate."
)

# Sex at Birth

In [ ]:
# Number of respondents that provided this datapoint
col = df[COLUMNS[0]]
responses = col.dropna().size
print(f"{responses} faculty answered this question for a {calcPercent(responses)}% response rate.")

In [ ]:
# Chart counts and percentages
col.fillna(value='No Answer', inplace=True)
# Count the occurences of each unique value in the column
col = col.value_counts()

# Create a table with the unique values as the row names
tmpDf = col.rename("Count").to_frame()
# Calculate how many did not answer this question
tmpDf.at['No Answer', 'Count'] = INVITED - tmpDf.iloc[:2, 0].sum()
# Calculate percentages for each unique value
tmpDf['Percent'] = calcPercent(tmpDf['Count'])
tmpDf.to_excel(DUMP, 'Sex')
tmpDf

In [ ]:
plt.bar(
    x=tmpDf.index,
    height=tmpDf['Count'],
    color=BAR_COLORS
)
plt.title(COLUMNS[0], fontsize=18)

In [ ]:
del responses, col, tmpDf

# Gender

In [ ]:
# Number of respondents that provided this datapoint
col = df[COLUMNS[1]]
responses = col.dropna().size
print(f"{responses} faculty answered this question for a {calcPercent(responses)}% response rate.")

In [ ]:
# Chart counts and percentages
col.fillna(value='No Answer', inplace=True)
col = col.value_counts()

tmpDf = col.rename("Count").to_frame()
tmpDf.at['No Answer', 'Count'] = INVITED - tmpDf.iloc[:2, 0].sum() - tmpDf.iloc[3:, 0].sum()
tmpDf['Percent'] = calcPercent(tmpDf['Count'])
tmpDf.to_excel(DUMP, 'Gender')
tmpDf

In [ ]:
fig = plt.figure(figsize=(15,10))
axes = fig.add_subplot()
axes.set_title(COLUMNS[1], fontsize=18)
plot = axes.bar(
    x=tmpDf.index,
    height=tmpDf['Count'],
    color=BAR_COLORS
)

In [ ]:
del responses, col, tmpDf, fig, axes, plot

# URIM

In [ ]:
# Select URIM columns
tmpDf = df.iloc[:, 2:9]

In [ ]:
# Count up how many survey respondents supplied URIM data
tmpDf['answered'] = 0
for i in range(2, 9):
    # For each URIM column, select non-null responses and mark them in the 'answered' column
    tmpDf.loc[~tmpDf[COLUMNS[i]].isnull(), 'answered'] = 1

responses = tmpDf[tmpDf.answered == 1].shape[0]
print(f"{responses} faculty answered the URIM question " + \
      f"for a {calcPercent(responses)}% response rate.")

# Remove rows with no URIM data
tmpDf.drop(tmpDf[tmpDf.answered == 0].index, inplace=True)
# Cleanup work column
tmpDf.drop('answered', axis=1, inplace=True)

In [ ]:
# Define and populate count table for each demographic, bearing in mind that
# multiple ethnicities can be selected by a single respondent
countDf = pd.DataFrame(columns=['Count', 'Percent'])
for i in range(2, 9):
    count = len(tmpDf[~tmpDf[COLUMNS[i]].isnull()])
    countDf.loc[COLUMNS[i]] = [
        count,
        calcPercent(count)
    ]

countDf.loc['No Answer', 'Count'] = INVITED - responses
countDf.loc['No Answer', 'Percent'] = calcPercent(countDf.at['No Answer', 'Count'])


countDf['Count'] = countDf['Count'].astype(int)
countDf.to_excel(DUMP, 'URiM')
countDf

"Prefer not to answer" and "No answer" are kept separate because they represent different nuances.  The former represents respondents who were willing to participate to at least some degree, while the latter represents respondents who chose not to answer at all.

In [ ]:
del responses, tmpDf, count, countDf

# Orientation

In [ ]:
# Number of respondents that provided this datapoint
col = df[COLUMNS[9]]
responses = col.dropna().size
print(f"{responses} faculty answered this question for a {calcPercent(responses)}% response rate.")

In [ ]:
# Chart counts and percentages
col.fillna(value='No Answer', inplace=True)
col = col.value_counts()

tmpDf = col.rename("Count").to_frame()
tmpDf.at['No Answer', 'Count'] = INVITED - tmpDf.iloc[:2, 0].sum() - tmpDf.iloc[3:, 0].sum()
tmpDf['Percent'] = calcPercent(tmpDf['Count'])
tmpDf.to_excel(DUMP, 'Orientation')
tmpDf

In [ ]:
fig = plt.figure(figsize=(15,10))
axes = fig.add_subplot()
axes.set_title(COLUMNS[9], fontsize=18)
plot = axes.bar(
    x=tmpDf.index,
    height=tmpDf['Count'],
    color=BAR_COLORS
)

In [ ]:
del responses, col, tmpDf, fig, axes, plot

# Accomodations

In [ ]:
# Number of respondents that provided this datapoint
col = df[COLUMNS[10]]
responses = col.dropna().size
print(f"{responses} faculty answered this question for a {calcPercent(responses)}% response rate.")

In [ ]:
# Chart counts and percentages
col.fillna(value='No Answer', inplace=True)
col = col.value_counts()

tmpDf = col.rename("Count").to_frame()
tmpDf.at['No Answer', 'Count'] = INVITED - responses
tmpDf['Percent'] = calcPercent(tmpDf['Count'])
tmpDf.to_excel(DUMP, 'Handicap')
tmpDf

In [ ]:
fig = plt.figure(figsize=(15,10))
axes = fig.add_subplot()
axes.set_title(COLUMNS[10], fontsize=18)
plot = axes.bar(
    x=tmpDf.index,
    height=tmpDf['Count'],
    color=BAR_COLORS
)

In [ ]:
del responses, col, tmpDf, fig, axes, plot

# Retirement

In [ ]:
# Number of respondents that provided this datapoint
col = df[COLUMNS[11]]
responses = col.dropna().size
print(f"{responses} faculty answered this question for a {calcPercent(responses)}% response rate.")

In [ ]:
# Chart counts and percentages
col.fillna(value='No Answer', inplace=True)
col = col.value_counts()

tmpDf = col.rename("Count").to_frame()
tmpDf.at['No Answer', 'Count'] = INVITED - responses
tmpDf['Percent'] = calcPercent(tmpDf['Count'])
tmpDf.to_excel(DUMP, 'Retirement')
tmpDf

In [ ]:
fig = plt.figure(figsize=(15,10))
axes = fig.add_subplot()
axes.set_title(COLUMNS[11], fontsize=18)
plot = axes.bar(
    x=tmpDf.index,
    height=tmpDf['Count'],
    color=BAR_COLORS
)

In [ ]:
del responses, col, tmpDf, fig, axes, plot

# Degrees

In [ ]:
# Select degree columns
tmpDf = df.iloc[:, 13:26]

In [ ]:
# Count up how many survey respondents supplied degree data
tmpDf['answered'] = 0
for i in range(13, 26):
    # For each degree column, select non-null responses and mark them in the 'answered' column
    tmpDf.loc[~tmpDf[COLUMNS[i]].isnull(), 'answered'] = 1

responses = tmpDf[tmpDf.answered == 1].shape[0]
print(f"{responses} faculty provide their degrees " + \
      f"for a {calcPercent(responses)}% response rate.")

# Remove rows with no degree data
tmpDf.drop(tmpDf[tmpDf.answered == 0].index, inplace=True)
# Cleanup work column
tmpDf.drop('answered', axis=1, inplace=True)

In [ ]:
# Define and populate count table for each degree group, bearing in mind that
# multiple degrees can be selected by a single respondent
countDf = pd.DataFrame(columns=['Count', 'Percent'])

# Clinical degrees
tmpDf['Counter'] = tmpDf[COLUMNS[13]].str.cat(tmpDf[COLUMNS[14:16]], na_rep='')
countDf.loc['Clinical', 'Count'] = tmpDf[tmpDf['Counter'].str.len() > 0].shape[0]

# Non-clinical degrees
tmpDf['Counter'] = tmpDf[COLUMNS[16]].str.cat(tmpDf[COLUMNS[17:19]], na_rep='')
countDf.loc['Non-clinical', 'Count'] = tmpDf[tmpDf['Counter'].str.len() > 0].shape[0]

# Other degrees
tmpDf['Counter'] = tmpDf[COLUMNS[19]].str.cat(tmpDf[COLUMNS[20:26]], na_rep='')
countDf.loc['Other', 'Count'] = tmpDf[tmpDf['Counter'].str.len() > 0].shape[0]

# No degrees given
countDf.loc['No Answer', 'Count'] = INVITED - responses

# Calculate percentages
countDf['Percent'] = countDf['Count'].apply(calcPercent)
countDf.to_excel(DUMP, 'Degrees')
countDf

In [ ]:
fig = plt.figure(figsize=(15,10))
axes = fig.add_subplot()
axes.set_title('Degrees', fontsize=18)
plot = axes.bar(
    x=countDf.index,
    height=countDf['Count'],
    color=BAR_COLORS
)

In [ ]:
del responses, tmpDf, countDf, fig, axes, plot

<hr style="border:3px solid gray">

# Setup - Pivot Base

In [ ]:
# Suppress SettingWithCopyWarning since we are using the .loc indexing method instead of the [][]
# chain indexing method it warns against.
# pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
#pd.options.mode.chained_assignment = <'warn' || 'raise' || None>

# Add rows representing those who didn't respond to the ACC using Sex at Birth as anchor column
df[COLUMNS[0]].fillna('No Answer', inplace=True)
for i in range(0, (INVITED - len(df))):
    df.loc[len(df), COLUMNS[0]] = 'No Answer'

# Fill in empty values in other categorical columns
for i in [1, 9, 10, 11]:
    df[COLUMNS[i]].fillna('No Answer', inplace=True)

# Replace str values in multi-select column groups with boolean values
# URIM Ethnicities
for i in range(2, 9):
    df.loc[df[COLUMNS[i]].notna(), COLUMNS[i]] = 1
    df[COLUMNS[i]].fillna(0, inplace=True)
# Note rows that have not answered the URIM question
df['urmNoAnswer'] = df.iloc[:, 2:9].sum(axis=1).apply(lambda x: 1 if x == 0 else 0)
COLUMNS += ['urmNoAnswer']

# Degrees
# List of translated degree column names
COLS_DEGR = [
    'degNoAnswer',
    'degClinical',
    'degNonClinical',
    'degOther'
]
COLUMNS += COLS_DEGR
# Concatenate strings of different degree categories together then convert the combined strings
# to boolean
def isMember(x):
    return 1 if len(x) > 0 else 0
df[COLS_DEGR[1]] = df[COLUMNS[13]].str.cat(df[COLUMNS[14:16]], na_rep='').apply(isMember)
df[COLS_DEGR[2]] = df[COLUMNS[16]].str.cat(df[COLUMNS[17:19]], na_rep='').apply(isMember)
df[COLS_DEGR[3]] = df[COLUMNS[19]].str.cat(df[COLUMNS[20:26]], na_rep='').apply(isMember)
# Add degree category that represents those who did not provide degree data
df[COLS_DEGR[0]] = (df.iloc[:, -3] + df.iloc[:, -2] + df.iloc[:, -1]) \
.apply(lambda x: 1 if x == 0 else 0)

# After this setup, new, transformed columns will have been added to the end of the dataframe:
# -5 = urmNoAnswer
# -4 = degClinical
# -3 = degNonClinical
# -2 = degOther
# -1 = degNoAnswer

# Assemble list of URIM column names
COLS_URIM = []
for i in ([-5] + list(range(2, 9))):
    COLS_URIM.append(df.iloc[:1, i].name)

del i, isMember

<hr style="border:3px solid gray">

# HMS Promotion Interest

## Setup

In [ ]:
# Number of respondents that provided this datapoint
responses = df[COLUMNS[12]].notna().sum()
print(f"{responses} faculty answered this question for a {calcPercent(responses)}% response rate.")

# Fill in NaN values in target column
df[COLUMNS[12]].fillna(value='No Answer', inplace=True)

# Define order that target column's values will be listed in in the pivot table
COLUMN_ORDER = ['No Answer', 'Yes', 'No', 'Total']

del responses

## Overall

In [ ]:
# Chart counts and percentages
col = df[COLUMNS[12]].value_counts()

tmpDf = col.rename("Count").to_frame()
tmpDf['Percent'] = calcPercent(tmpDf['Count'])
tmpDf.to_excel(DUMP, 'HmsPromoInterest')
tmpDf

In [ ]:
fig = plt.figure(figsize=(15,10))
axes = fig.add_subplot()
axes.set_title(COLUMNS[12], fontsize=18)
plot = axes.bar(
    x=tmpDf.index,
    height=tmpDf['Count'],
    color=BAR_COLORS
)

In [ ]:
del tmpDf, fig, axes, plot

## Grouped by Sex at Birth

In [ ]:
pivot = pd.crosstab(
    df[COLUMNS[0]], df[COLUMNS[12]],
    margins=True,
    margins_name='Total'
)
# Reorder indices and columns
pivot = pivot.loc[
    ['No Answer', 'Female', 'Male', 'Total'],
    COLUMN_ORDER
]
pivot.to_excel(DUMP, 'HmsPromoBySex')
pivot

### Example reading:

Of those who did not indicate either way on HMS promotions:
 - 273 did not indicate their sex at birth
 - 12 were female at birth
 - 12 were male at birth

Of those who expressed interest in HMS promotion:
 - 5 did not provide their sex at birth
 - 244 were female at birth
 - 216 were male at birth

Of those who explicitly did not express interest in HMS promotion:
 - 3 did not provide their sex at birth
 - 166 were female at birth
 - 164 were male at birth

### Example of calculating percentages for a given category

In [ ]:
# Example percentage calculation for a given category
category = pivot.loc['Female']
total = category.pop('Total')
category.apply(calcPercent, denominator=total)

In [ ]:
del pivot, category, total

## Grouped by Gender

In [ ]:
pivot = pd.crosstab(
    df[COLUMNS[1]], df[COLUMNS[12]],
    margins=True,
    margins_name='Total'
)
# Reorder indices and columns
pivot = pivot.loc[
    ['No Answer', 'Woman', 'Man', 'Prefer not to answer', 'Nonbinary', 'Total'],
    COLUMN_ORDER
]
pivot.to_excel(DUMP, 'HmsPromoByGender')
pivot

In [ ]:
del pivot

## Grouped by URIM Status

In [ ]:
# Assemble crosstabbed rows
rows = []
for col in COLS_URIM:
    pivot = pd.crosstab(
        df.loc[df[col] == 1, col],
        df[COLUMNS[12]],
        margins=True,
        margins_name='Total'
    )
    
    # Check for and add unrepresented HMS Promo Question answers to current category
    row = pivot.loc[1].rename(col)
    for answer in COLUMN_ORDER:
        if answer not in row.index:
            row.at[answer] = 0
    
    # Append crosstabbed series to list
    rows.append(row)

<b>Interest in HMS Promotion by URIM Identity</b>

In [ ]:
tmpDf = pd.DataFrame(rows, columns=COLUMN_ORDER)
tmpDf.to_excel(DUMP, 'HmsPromoByURiM')
tmpDf

In [ ]:
del rows, col, pivot, row, answer, tmpDf

## Grouped by Orientation

In [ ]:
pivot = pd.crosstab(
    df[COLUMNS[9]], df[COLUMNS[12]],
    margins=True,
    margins_name='Total'
)
# Reorder indices and columns
pivot = pivot.loc[
    [
        'No Answer',
        'Prefer not to answer',
        'Straight/Heterosexual',
        'Gay',
        'Lesbian',
        'Queer',
        'Bisexual',
        'Other'
    ],
    COLUMN_ORDER
]
pivot.to_excel(DUMP, 'HmsPromoByOrientation')
pivot

In [ ]:
del pivot

## Grouped by Need for Accommodations

In [ ]:
pivot = pd.crosstab(
    df[COLUMNS[10]], df[COLUMNS[12]],
    margins=True,
    margins_name='Total'
)
# Reorder indices and columns
pivot = pivot.loc[
    ['No Answer', 'Yes', 'No'],
    COLUMN_ORDER
]
pivot.to_excel(DUMP, 'HmsPromoByHandicap')
pivot

In [ ]:
del pivot

## Grouped by Retirement Plans

In [ ]:
pivot = pd.crosstab(
    df[COLUMNS[11]], df[COLUMNS[12]],
    margins=True,
    margins_name='Total'
)
# Reorder indices and columns
pivot = pivot.loc[
    ['No Answer', 'Yes', 'No'],
    COLUMN_ORDER
]
pivot.to_excel(DUMP, 'HmsPromoByRetirement')
pivot

In [ ]:
del pivot

## Grouped by Degree Category

In [ ]:
# Assemble crosstabbed rows
rows = []
for col in COLS_DEGR:
    pivot = pd.crosstab(
        df.loc[df[col] == 1, col],
        df[COLUMNS[12]],
        margins=True,
        margins_name='Total'
    )
    
    # Check for and add unrepresented HMS Promo Question answers to current category
    row = pivot.loc[1].rename(col)
    for answer in COLUMN_ORDER:
        if answer not in row.index:
            row.at[answer] = 0
    
    # Append crosstabbed series to list
    rows.append(row)

<b>Interest in HMS Promotion by Degree Category</b>

In [ ]:
tmpDf = pd.DataFrame(rows, columns=COLUMN_ORDER)
tmpDf.to_excel(DUMP, 'HmsPromoByDegree')
tmpDf

In [ ]:
del rows, col, pivot, row, answer, COLUMN_ORDER, tmpDf

<hr style="border:3px solid gray">

# HMS Rank

## Setup

In [ ]:
# Set 'N/A...' strings to nulls
for i in range(34, 27, -2):
    df.loc[df[COLUMNS[i]] == 'N/A - previous is my current appointment', COLUMNS[i]] = pd.NA

# Determine current HMS appointment
# Appointments are listed from left to right in chronological order
# Work in reverse column order to determine current appointment
def getCurrent(row):
    for i in range(34, 25, -2):
        if pd.notna(row.loc[COLUMNS[i]]):
            return row.loc[COLUMNS[i]]
    return pd.NA

TARGET = 'hmsCurrent'

df[TARGET] = df.apply(getCurrent, axis=1)
COLUMNS += [TARGET]

responses = df[TARGET].notna().sum()
print(f"{responses} faculty answered this question for a {calcPercent(responses)}% response rate.")

# Fill in NaN values in target column
df[TARGET].fillna(value='No Answer', inplace=True)

# Define order that target column's values will be listed in in the pivot table
COLUMN_ORDER = [
    'No Answer',
    'Instructor',
    'Assistant Professor',
    'Associate Professor',
    'Full Professor',
    'Research Associate',
    'member of the faculty',
    'Total'
]

del i, getCurrent, responses

## Overall

In [ ]:
# Chart counts and percentages
col = df[TARGET].value_counts()

tmpDf = col.rename("Count").to_frame()
tmpDf['Percent'] = calcPercent(tmpDf['Count'])
tmpDf.to_excel(DUMP, 'HmsRank')
tmpDf

In [ ]:
fig = plt.figure(figsize=(15,10))
axes = fig.add_subplot()
axes.set_title('Current HMS Rank', fontsize=18)
plot = axes.bar(
    x=tmpDf.index,
    height=tmpDf['Count'],
    color=BAR_COLORS
)

In [ ]:
del tmpDf, fig, axes, plot

## Grouped by Sex at Birth

In [ ]:
pivot = pd.crosstab(
    df[COLUMNS[0]], df[TARGET],
    margins=True,
    margins_name='Total'
)
# Reorder indices and columns
pivot = pivot.loc[
    ['No Answer', 'Female', 'Male', 'Total'],
    COLUMN_ORDER
]
pivot.to_excel(DUMP, 'HmsRankBySex')
pivot

In [ ]:
del pivot

## Grouped by Gender

In [ ]:
pivot = pd.crosstab(
    df[COLUMNS[1]], df[TARGET],
    margins=True,
    margins_name='Total'
)
# Reorder indices and columns
pivot = pivot.loc[
    ['No Answer', 'Woman', 'Man', 'Prefer not to answer', 'Nonbinary', 'Total'],
    COLUMN_ORDER
]
pivot.to_excel(DUMP, 'HmsRankByGender')
pivot

In [ ]:
del pivot

## Grouped by URIM Status

In [ ]:
# Assemble crosstabbed rows
rows = []
for col in COLS_URIM:
    pivot = pd.crosstab(
        df.loc[df[col] == 1, col],
        df[TARGET],
        margins=True,
        margins_name='Total'
    )
    
    # Check for and add unrepresented answers to current category
    row = pivot.loc[1].rename(col)
    for answer in COLUMN_ORDER:
        if answer not in row.index:
            row.at[answer] = 0
    
    # Append crosstabbed series to list
    rows.append(row)

<b>HMS Rank by URIM Identity</b>

In [ ]:
tmpDf = pd.DataFrame(rows, columns=COLUMN_ORDER)
tmpDf.to_excel(DUMP, 'HmsRankByURiM')
tmpDf

In [ ]:
del rows, col, pivot, row, answer, tmpDf

## Grouped by Orientation

In [ ]:
pivot = pd.crosstab(
    df[COLUMNS[9]], df[TARGET],
    margins=True,
    margins_name='Total'
)
# Reorder indices and columns
pivot = pivot.loc[
    [
        'No Answer',
        'Prefer not to answer',
        'Straight/Heterosexual',
        'Gay',
        'Lesbian',
        'Queer',
        'Bisexual',
        'Other'
    ],
    COLUMN_ORDER
]
pivot.to_excel(DUMP, 'HmsRankByOrientation')
pivot

In [ ]:
del pivot

## Grouped by Need for Accommodations

In [ ]:
pivot = pd.crosstab(
    df[COLUMNS[10]], df[TARGET],
    margins=True,
    margins_name='Total'
)
# Reorder indices and columns
pivot = pivot.loc[
    ['No Answer', 'Yes', 'No'],
    COLUMN_ORDER
]
pivot.to_excel(DUMP, 'HmsRankByHandicap')
pivot

In [ ]:
del pivot

## Grouped by Retirement Plans

In [ ]:
pivot = pd.crosstab(
    df[COLUMNS[11]], df[TARGET],
    margins=True,
    margins_name='Total'
)
# Reorder indices and columns
pivot = pivot.loc[
    ['No Answer', 'Yes', 'No'],
    COLUMN_ORDER
]
pivot.to_excel(DUMP, 'HmsRankByRetirement')
pivot

In [ ]:
del pivot

## Grouped by Degree Category

In [ ]:
# Assemble crosstabbed rows
rows = []
for col in COLS_DEGR:
    pivot = pd.crosstab(
        df.loc[df[col] == 1, col],
        df[TARGET],
        margins=True,
        margins_name='Total'
    )
    
    # Check for and add unrepresented HMS Promo Question answers to current category
    row = pivot.loc[1].rename(col)
    for answer in COLUMN_ORDER:
        if answer not in row.index:
            row.at[answer] = 0
    
    # Append crosstabbed series to list
    rows.append(row)

<b>HMS Rank by Degree Category</b>

In [ ]:
tmpDf = pd.DataFrame(rows, columns=COLUMN_ORDER)
tmpDf.to_excel(DUMP, 'HmsRankByDegree')
tmpDf

In [ ]:
del rows, col, pivot, row, answer, tmpDf

In [ ]:
del COLUMN_ORDER, TARGET

<hr style="border:3px solid gray">

# MGH Rank

## Setup

In [ ]:
# Set 'N/A...' strings to nulls
for i in range(44, 35, -2):
    df.loc[df[COLUMNS[i]] == 'N/A - previous is my current appointment', COLUMNS[i]] = pd.NA

# Determine current MGH appointment
# Appointments are listed from left to right in chronological order
# Work in reverse column order to determine current appointment
# Unlike HMS appointments, some respondents answered N/A in the left-most MGH title
def getCurrent(row):
    for i in range(44, 35, -2):
        if pd.notna(row.loc[COLUMNS[i]]):
            return row.loc[COLUMNS[i]]
    return pd.NA

TARGET = 'mghCurrent'
COLUMNS += [TARGET]

df[TARGET] = df.apply(getCurrent, axis=1)

responses = df[TARGET].notna().sum()
print(f"{responses} staff answered this question for a {calcPercent(responses)}% response rate.")

# Fill in NaN values in target column
df[TARGET].fillna(value='No Answer', inplace=True)

# Define order that target column's values will be listed in in the pivot table
COLUMN_ORDER = [
    'No Answer',
    'Research Staff',
    'Assistant in Medicine',
    'Assistant Physician',
    'Associate Physician',
    'Physician',
    'Clinical Affiliate',
    'Clinical Associate',
    'Clinical Assistant',
    'Other',
    'Total'
]

del i, getCurrent, responses

## Overall

In [ ]:
# Chart counts and percentages
col = df[TARGET].value_counts()

tmpDf = col.rename("Count").to_frame()
tmpDf['Percent'] = calcPercent(tmpDf['Count'])
tmpDf.to_excel(DUMP, 'MghRank')
tmpDf

In [ ]:
fig = plt.figure(figsize=(15,10))
axes = fig.add_subplot()
axes.set_title('Current MGH Rank', fontsize=18)
plot = axes.bar(
    x=tmpDf.index,
    height=tmpDf['Count'],
    color=BAR_COLORS
)

In [ ]:
del tmpDf, fig, axes, plot

## Grouped by Sex at Birth

In [ ]:
pivot = pd.crosstab(
    df[COLUMNS[0]], df[TARGET],
    margins=True,
    margins_name='Total'
)
# Reorder indices and columns
pivot = pivot.loc[
    ['No Answer', 'Female', 'Male', 'Total'],
    COLUMN_ORDER
]
pivot.to_excel(DUMP, 'MghRankBySex')
pivot

In [ ]:
del pivot

## Grouped by Gender

In [ ]:
pivot = pd.crosstab(
    df[COLUMNS[1]], df[TARGET],
    margins=True,
    margins_name='Total'
)
# Reorder indices and columns
pivot = pivot.loc[
    ['No Answer', 'Woman', 'Man', 'Prefer not to answer', 'Nonbinary', 'Total'],
    COLUMN_ORDER
]
pivot.to_excel(DUMP, 'MghRankByGender')
pivot

In [ ]:
del pivot

## Grouped by URIM Status

In [ ]:
# Assemble crosstabbed rows
rows = []
for col in COLS_URIM:
    pivot = pd.crosstab(
        df.loc[df[col] == 1, col],
        df[TARGET],
        margins=True,
        margins_name='Total'
    )
    
    # Check for and add unrepresented answers to current category
    row = pivot.loc[1].rename(col)
    for answer in COLUMN_ORDER:
        if answer not in row.index:
            row.at[answer] = 0
    
    # Append crosstabbed series to list
    rows.append(row)

<b>MGH Rank by URIM Identity</b>

In [ ]:
tmpDf = pd.DataFrame(rows, columns=COLUMN_ORDER)
tmpDf.to_excel(DUMP, 'MghRankByURiM')
tmpDf

In [ ]:
del rows, col, pivot, row, answer, tmpDf

## Grouped by Orientation

In [ ]:
pivot = pd.crosstab(
    df[COLUMNS[9]], df[TARGET],
    margins=True,
    margins_name='Total'
)
# Reorder indices and columns
pivot = pivot.loc[
    [
        'No Answer',
        'Prefer not to answer',
        'Straight/Heterosexual',
        'Gay',
        'Lesbian',
        'Queer',
        'Bisexual',
        'Other'
    ],
    COLUMN_ORDER
]
pivot.to_excel(DUMP, 'MghRankByOrientation')
pivot

In [ ]:
del pivot

## Grouped by Need for Accommodations

In [ ]:
pivot = pd.crosstab(
    df[COLUMNS[10]], df[TARGET],
    margins=True,
    margins_name='Total'
)
# Reorder indices and columns
pivot = pivot.loc[
    ['No Answer', 'Yes', 'No'],
    COLUMN_ORDER
]
pivot.to_excel(DUMP, 'MghRankByHandicap')
pivot

In [ ]:
del pivot

## Grouped by Retirement Plans

In [ ]:
pivot = pd.crosstab(
    df[COLUMNS[11]], df[TARGET],
    margins=True,
    margins_name='Total'
)
# Reorder indices and columns
pivot = pivot.loc[
    ['No Answer', 'Yes', 'No'],
    COLUMN_ORDER
]
pivot.to_excel(DUMP, 'MghRankByRetirement')
pivot

In [ ]:
del pivot

## Grouped by Degree Category

In [ ]:
# Assemble crosstabbed rows
rows = []
for col in COLS_DEGR:
    pivot = pd.crosstab(
        df.loc[df[col] == 1, col],
        df[TARGET],
        margins=True,
        margins_name='Total'
    )
    
    # Check for and add unrepresented HMS Promo Question answers to current category
    row = pivot.loc[1].rename(col)
    for answer in COLUMN_ORDER:
        if answer not in row.index:
            row.at[answer] = 0
    
    # Append crosstabbed series to list
    rows.append(row)

<b>MGH Rank by Degree Category</b>

In [ ]:
tmpDf = pd.DataFrame(rows, columns=COLUMN_ORDER)
tmpDf.to_excel(DUMP, 'MghRankByDegree')
tmpDf

In [ ]:
del rows, col, pivot, row, answer, tmpDf

In [ ]:
del COLUMN_ORDER, TARGET

<hr style="border:3px solid gray">

# Date Conversions

In [ ]:
def getType(x):
    return pd.NA if pd.isna(x) else type(x)

for i in range(27,46,2):
    print(f"{i} {COLUMNS[i]}\n{df[COLUMNS[i]].apply(getType).unique()}\n")

del getType, i

Respondents were allowed to enter whatever string they wished into these "date" fields.  Some responses even include typos such as missing or extra digits.

Ranks were also allowed to be chosen from a list without restricting for a coherent sense of progression.  Ranks could be repeated multiple times or listed in any order.

* Only those responses that can be read as a coherent, single year value will be evaluated.
* Any non-numerical characters will invalidate a response.
* Year values that do not fall within a sensible range will be invalidated.
* All dates will be assumed to be start dates.
* Ranks repeated consecutively will be considered the same appointment with no breaks in between.  The earliest year will be used for calculations.
* Any difference in titles will be considered a "promotion" whether or not there is an actual progression in the ranks.

## Setup

In [ ]:
# Create Promotion Columns
df['hmsFirstLength'] = pd.NA
COLUMNS += ['hmsFirstLength']
for i in range(0, 5):
    s = str(i)
    df['hmsPromoType' + s] = pd.NA
    COLUMNS += ['hmsPromoType' + s]
    df['hmsPromoLength' + s] = pd.NA
    COLUMNS += ['hmsPromoLength' + s]
df['mghFirstLength'] = pd.NA
COLUMNS += ['mghFirstLength']
for i in range(0, 5):
    s = str(i)
    df['mghPromoType' + s] = pd.NA
    COLUMNS += ['mghPromoType' + s]
    df['mghPromoLength' + s] = pd.NA
    COLUMNS += ['mghPromoLength' + s]

# Helper that defines valid year range
def inRange(year):
    return (year > 1949 and year <= datetime.now().year)

# Helper that converts various data types to integer years
def getYear(x):
    yr = pd.NA
    # Match-Case Syntax wasn't introduced until Python 3.10
    # This document was written with Python 3.9
    if isinstance(x, int):
        if inRange(x):
            yr = x
    elif isinstance(x, datetime):
        if inRange(x.year):
            yr = x.year
    elif isinstance(x, float) and pd.notna(x):
        if inRange(int(x)):
            yr = int(x)
    return yr

# grp = 'hms' || 'mgh'
def calcPromos(row, grp):
    ranks = range(26, 35, 2) if (grp == 'hms') else range(36, 45, 2)
    prvTitle = None
    prvStart = None
    promoNum = 0
    for i in range(5):
        nxtTitle = row[COLUMNS[ranks[i]]]
        nxtStart = getYear(row[COLUMNS[ranks[i]+1]])
        
        # Stop if there is not enough data to calculate length between ranks
        if pd.isna(nxtTitle) or pd.isna(nxtStart):
            break
        
        if prvTitle == None:
            prvTitle = nxtTitle
            prvStart = nxtStart
            continue
        elif prvTitle != nxtTitle:
            # Calculate and record time between promotion
            length = nxtStart - prvStart
            if length >= 0:
                row[grp + 'PromoType' + str(promoNum)] = prvTitle + ' to ' + nxtTitle
                row[grp + 'PromoLength' + str(promoNum)] = length
            else:
                row[grp + 'PromoType' + str(promoNum)] = nxtTitle + ' to ' + prvTitle
                row[grp + 'PromoLength' + str(promoNum)] = length * -1
            prvTitle = nxtTitle
            prvStart = nxtStart
            promoNum += 1
        elif prvTitle == nxtTitle:
            # Account for appointments listed in descending order
            if nxtStart < prvStart:
                prvStart = nxtStart
    
    # Calculate time spent at initial rank for those who weren't promoted
    # or the final listed rank of those who were.
    # Assumes there has been no break in service
    if pd.notna(prvTitle) and pd.notna(prvStart):
        row[grp + 'PromoType' + str(promoNum)] = prvTitle + '_'
        row[grp + 'PromoLength' + str(promoNum)] = datetime.now().year - prvStart
    
    return row

# Apply algorithm
df = df.apply(calcPromos, axis=1, args=('hms',))
df = df.apply(calcPromos, axis=1, args=('mgh',))

del i, inRange, getYear, calcPromos

<hr style="border:3px solid gray">

# Close and Save Data Dump Excel File

In [ ]:
# Reorder columns
COLUMNS = COLUMNS[:9] + [COLUMNS[46]] + COLUMNS[9:13] + COLUMNS[47:51] + COLUMNS[13:26] + \
[COLUMNS[51]] + COLUMNS[53:61] + COLUMNS[26:36] + [COLUMNS[52]] + COLUMNS[61:] + \
COLUMNS[36:46]

df.to_excel(DUMP, sheet_name='Source', columns=COLUMNS)
DUMP.close()